In [2]:
import pymongo
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import time
from datetime import date
from datetime import timedelta
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import json
import requests
from pymongo.errors import BulkWriteError
from pprint import pprint
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import emoji
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from itertools import chain
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# if 'Cleaning_status' which is 'compete' it will update it into 'pending'

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['Final_Year_Project']
collection = db[f'stock_twits_dataset']
collection.update_many({'Cleaning_status':'complete'},{ '$set': { 'Cleaning_status': "pending" }})

In [9]:
",".join(stopwords.words('english'))
stop_words = set(stopwords.words('english'))
print(stop_words)
def remove_stopwords(x):
    return " ".join([word for word in str(x).split() if word not in stop_words])

{'your', 'their', 'me', 'from', 'her', 'only', "couldn't", 'his', 'yours', 'ma', 'weren', 'of', 'hadn', "hadn't", 'through', "don't", 'no', 'during', 'ourselves', 'more', 'nor', 'ours', 'not', 'each', "that'll", "doesn't", 'below', 'i', "it's", 'on', 'd', 'when', 'having', 'she', "mightn't", 'up', 'don', 'we', 'they', 'whom', 're', "shouldn't", 'for', 'was', 'hasn', 'most', "she's", 'few', 'above', "haven't", 'wasn', 'do', 'wouldn', 'out', 'as', 'very', 'll', 'who', 'its', 'it', 'the', 'or', 'where', 'myself', 'him', 'are', 'because', 'won', "wouldn't", 'yourselves', 'down', 'm', 'own', 'been', 'hers', 'doing', 'before', 'them', 't', 'now', 'over', 'were', "wasn't", "aren't", "hasn't", 'will', 'herself', 'just', 'but', 'our', 'again', 'so', 'those', 'a', 'after', 've', "didn't", "you've", 'am', 'that', 'against', "you'd", 'themselves', 'into', 'such', 'he', 'what', 'between', 'under', "mustn't", 'o', "shan't", 'any', 'you', 'in', 'doesn', 'then', 'isn', 'mightn', 'about', 'theirs', 'at

# Actual Code for stemming and cleaning

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['Final_Year_Project']
collection = db[f'stock_twits_dataset']
collection_stemming = db[f'stock_twits_dataset_stemming']
# While True :
status = 'pending'

    # Take While loop here

while (True):
    records = collection.find({'Cleaning_status': status}).limit(1000)
    # We can change the limit of records from here
    id_list          = []
    twits_list       = []
    signal_list      = []
    stock_name_list  = [] 
    date_list        = []
#     index_number = 0  # The purpose of this variable is to traverse each row in dataframe
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    stop_words = set(stopwords.words('english'))
    def remove_stopwords(x):
        return " ".join([word for word in str(x).split() if word not in stop_words])   # it is in use
    def remove_punctuation(x):
        return " ".join([word for word in str(x).spit() if word not in punc ])
    for record in records:
        #     print(record)
        twit                 = record['Twits']
        signal               = record['Signal']
        stock_name           = record['stock_name']
        cleaning_status      = record['Cleaning_status']
        date                 = record['Date']
        id_name              = record['_id']

        # Adding all the 1000 TWITS in a list 
        twits_list.append(twit)

        # Adding all the 1000 SIGNALS in a list 
        signal_list.append(signal)

        # Adding all the 1000 Stock name in a list 
        stock_name_list.append(stock_name)

        # Adding all the 1000 date in a list 
        date_list.append(date)

        # Adding all the 1000 _id in a list 
        id_list.append(id_name)


    #Adding all columns in dataframe with there values
    df = pd.DataFrame(list(zip(twits_list,signal_list)),columns =['twits','signal'])
    df['stock_name'] = stock_name_list
    df['Date'] = date_list


    # resetting the index column 
    df.reset_index(inplace=True)


    #Removing Links from dataframe
    print('---------------------------------Links------------------------------------')
    df['twits'] = df['twits'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)



    #Lowercase the whole twits column
    print('---------------------------------lowercase------------------------------------')
    df['twits'] = df['twits'].apply(lambda x: x.lower())



    #Eliminating the pumctuation 
    print('----------------------------eliminate punctuation-------------------------------')
    #     translator = str.maketrans('', '', string.punctuation)
    #     df['twits'] = df['twits'].apply(lambda x: x.translate(translator))  # from every twits string.punctuation (!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~) should remove  
    df['twits'] = df['twits'].str.replace('[^\w\s]',' ',regex=True)



    #Eliminating stopwords 
    print('----------------------------removing stopwords-------------------------------')
    df['twits'] = df['twits'].apply(lambda x : remove_stopwords(x))



    #Eliminating Emoji from dataframe
    print('----------------------------removing emojis-------------------------------')
    df['twits'] = df['twits'].apply(lambda s: emoji.replace_emoji(s, ''))


    # Stemming on dataframe
    print('----------------------------stemming is done-------------------------------')
    stemmer = PorterStemmer()
    df['twits'] = df['twits'].str.split()
    df['twits'] = df['twits'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
    df['twits'] = df['twits'].apply(lambda x: " ".join(x))
    

    
    # Removing number str from column twits
    print('----------------------------removing number is done-------------------------------')
    df['twits'] = df['twits'].replace('\d+', '', regex=True)
    
    
    # Removing the words which contain only 1 letter
    print('----------------------------removing 1 letter word is done-------------------------------')
    df['twits'] = df['twits'].str.split()
    for text in df['twits']:
        for word in text:
            if len(word) == 1:
                text.remove(word)
    df['twits'] = df['twits'].apply(lambda x: " ".join(x))
    print('___________________________________________________________________________________________________________________')
    
    # For displaying purpose only
    print(len(twits_list))
    print(len(signal_list))


    #Converting dataframe into a list of dictionary
    documents = df.to_dict(orient='records')


    # Inserting the 100 stemming records in list of dictionary
    collection_stemming.insert_many(documents)
    print(f'inserting 100 records in database is done ')

    
    # Status completion in Stock_twits_dataset 
    for i in id_list:
        collection.update_many({'_id':i}, {'$set': {'Cleaning_status':'complete'}})
        #     status_initial = { '_id': i } 
        #     status_update = { "$set": {'Cleaning_status': 'complete'}}
        #     collection.update_many(status_initial,status_update)
    
    
    print('1000 done')
    print('#################################################Status updated in Stock_twits_dataset ###########################################################################')

---------------------------------Links------------------------------------
---------------------------------lowercase------------------------------------
----------------------------eliminate punctuation-------------------------------
----------------------------removing stopwords-------------------------------
----------------------------removing emojis-------------------------------
----------------------------stemming is done-------------------------------
----------------------------removing number is done-------------------------------
----------------------------removing 1 letter word is done-------------------------------
___________________________________________________________________________________________________________________
1000
1000
inserting 100 records in database is done 
1000 done
#################################################Status updated in Stock_twits_dataset ###########################################################################
---------------------

inserting 100 records in database is done 
1000 done
#################################################Status updated in Stock_twits_dataset ###########################################################################
---------------------------------Links------------------------------------
---------------------------------lowercase------------------------------------
----------------------------eliminate punctuation-------------------------------
----------------------------removing stopwords-------------------------------
----------------------------removing emojis-------------------------------
----------------------------stemming is done-------------------------------
----------------------------removing number is done-------------------------------
----------------------------removing 1 letter word is done-------------------------------
___________________________________________________________________________________________________________________
1000
1000
inserting 100 records

----------------------------removing number is done-------------------------------
----------------------------removing 1 letter word is done-------------------------------
___________________________________________________________________________________________________________________
1000
1000
inserting 100 records in database is done 
1000 done
#################################################Status updated in Stock_twits_dataset ###########################################################################
---------------------------------Links------------------------------------
---------------------------------lowercase------------------------------------
----------------------------eliminate punctuation-------------------------------
----------------------------removing stopwords-------------------------------
----------------------------removing emojis-------------------------------
----------------------------stemming is done-------------------------------
---------------------

TypeError: documents must be a non-empty list

In [11]:
# STEMMING IS DONE
for i in range(100):
    print(df['twits'][i])
    print('--------------')

tatasteel nse
--------------
tatasteel nse look like fake breakout stay cautiou
--------------
tatasteel nse trail stoploss 115
--------------
tatasteel nse long trade action safe trader exit around level 123
--------------
tatasteel nse cmp 119 80 anoth counter show uptrend recent saw bullish crossov 50 200 daili chart stock made high 124 75 time might break previou high 124 75 buy dip keep stock bullish mode
--------------
tatasteel nse look realli good daili chart consecut day form doji candl post give strong run buy start trade 121
--------------
tatasteel nse given breakout rise wedg pattern daili timefram breakout also given close 0 786 119 40 next expect target rs 124
--------------
tatasteel nse wit breakout
--------------
tatasteel nse 121
--------------
tatasteel nse standalon adjust ebitda inr65bn estim inr61bn 22 qoq season lower volum absenc one save record q4fy23 highlight adj standalon ebitda inr14 069 fall inr2 634 qoq ii europ ebitda loss stay high usd96 usd93 q4fy23 i

In [8]:
# STEMMING IS NOT DONE
for i in range(100):
    print(df['twits'][i])
    print('--------------')

['tatasteel', 'nse']
--------------
['tatasteel', 'nse', 'looks', 'like', 'fake', 'breakout', 'stay', 'cautious']
--------------
['tatasteel', 'nse', 'trail', 'stoploss', '115']
--------------
['tatasteel', 'nse', 'long', 'trade', 'action', 'safe', 'traders', 'exit', 'around', 'level', '123']
--------------
['tatasteel', 'nse', 'cmp', '119', '80', 'another', 'counter', 'showing', 'uptrend', 'recently', 'saw', 'bullish', 'crossover', '50', '200', 'daily', 'chart', 'stock', 'made', 'high', '124', '75', 'time', 'might', 'break', 'previous', 'high', '124', '75', 'buy', 'dip', 'keeping', 'stock', 'bullish', 'mode']
--------------
['tatasteel', 'nse', 'looking', 'really', 'good', 'daily', 'charts', 'consecutive', 'days', 'formed', 'doji', 'candle', 'post', 'giving', 'strong', 'run', 'buy', 'starts', 'trading', '121']
--------------
['tatasteel', 'nse', 'given', 'breakout', 'rising', 'wedge', 'pattern', 'daily', 'timeframe', 'breakout', 'also', 'given', 'closing', '0', '786', '119', '40', 'ne

In [7]:
display(df)

,Twits,Signals
0,$TATASTEEL.NSE,Bullish
1,$TATASTEEL.NSE looks like a fake breakout!! st...,Bullish
2,$TATASTEEL.NSE \nTrail the stoploss to 115.,Null
3,$TATASTEEL.NSE long trade in action \nsafe tr...,Null
4,$TATASTEEL.NSE CMP - 119.80 \nAnother counter...,Bullish
...,...,...
95,$TATASTEEL.NSE finally waking up!!! \nBreakout...,Bullish
96,$TATASTEEL.NSE Cleared lots of resistance zone...,Bullish
97,$TATASTEEL.NSE \nKeep in radar.. looks complet...,Null
98,Traders can go long in $TATASTEEL.NSE if it su...,Bullish


In [35]:
parse_twits_final = []
parse_twits = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
cleaned_data =[]
for i in range(len(df['Twits'])):
    patt  = r'\\n'
    rt = re.compile(patt) 
    _n_ = rt.sub('',df['Twits'][i])
    parse_twits.append(_n_)
for i in parse_twits:
    i = i.lower()
    parse_twits_final.append(i)
for i in parse_twits_final:
    for alphabet in i:
        if alphabet in punc:
            i = i.replace(alphabet, "")
    cleaned_data.append(i)


In [36]:
parse_twits

['$TATASTEEL.NSE',
 '$TATASTEEL.NSE looks like a fake breakout!! stay cautious.',
 '$TATASTEEL.NSE  \nTrail the stoploss to 115.',
 '$TATASTEEL.NSE long trade in action  \nsafe traders can exit around the level of 123',
 '$TATASTEEL.NSE  CMP - 119.80 \nAnother counter which is showing an uptrend and recently saw a bullish crossover of 50 MA over 200 MA and in the daily chart the stock made a high of 124.75 and this time might break the previous high of 124.75.  \nBUY THE DIP keeping the stock on bullish mode',
 '$TATASTEEL.NSE is looking really good on the daily charts. For the consecutive days it has formed a doji candle, post giving a strong run up. Buy it if starts trading above 121.',
 '$TATASTEEL.NSE has given a breakout from Rising wedge pattern on daily timeframe. \nAfter breakout, it has also given closing above 0.786(119.40). \nThe next expected target can be Rs.124.',
 '$TATASTEEL.NSE witnessing a BREAKOUT!!',
 '$TATASTEEL.NSE  \n \n121',
 '$TATASTEEL.NSE standalone adjusted 

In [14]:
twits_list

['$TATASTEEL.NSE',
 '$TATASTEEL.NSE looks like a fake breakout!! stay cautious.',
 '$TATASTEEL.NSE  \nTrail the stoploss to 115.',
 '$TATASTEEL.NSE long trade in action  \nsafe traders can exit around the level of 123',
 '$TATASTEEL.NSE  CMP - 119.80 \nAnother counter which is showing an uptrend and recently saw a bullish crossover of 50 MA over 200 MA and in the daily chart the stock made a high of 124.75 and this time might break the previous high of 124.75.  \nBUY THE DIP keeping the stock on bullish mode',
 '$TATASTEEL.NSE is looking really good on the daily charts. For the consecutive days it has formed a doji candle, post giving a strong run up. Buy it if starts trading above 121.',
 '$TATASTEEL.NSE has given a breakout from Rising wedge pattern on daily timeframe. \nAfter breakout, it has also given closing above 0.786(119.40). \nThe next expected target can be Rs.124.',
 '$TATASTEEL.NSE witnessing a BREAKOUT!!',
 '$TATASTEEL.NSE  \n \n121',
 '$TATASTEEL.NSE standalone adjusted 

In [15]:
signal_list

['Bullish',
 'Bullish',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Bullish',
 'Bearish',
 'Bullish',
 'Null',
 'Bullish',
 'Bearish',
 'Bearish',
 'Null',
 'Null',
 'Null',
 'Null',
 'Bullish',
 'Bullish',
 'Bullish',
 'Null',
 'Bullish',
 'Null',
 'Null',
 'Bearish',
 'Null',
 'Bullish',
 'Null',
 'Bearish',
 'Null',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Null',
 'Bearish',
 'Bearish',
 'Bearish',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Null',
 'Bearish',
 'Bullish',
 'Bullish',
 'Null',
 'Null',
 'Bullish',
 'Bullish',
 'Null',
 'Bullish',
 'Bearish',
 'Bullish',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Bullish',
 'Bearish',
 'Null',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Bearish',
 'Null',
 'Null',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Bullish',
 'Bullish',
 'Null',
 'Bullish',
 'Null',
 'Bullish',
 'Bu

In [16]:
# df = pd.DataFrame(twits_list)
df = pd.DataFrame(list(zip(twits_list,signal_list)),columns =['Twits', 'Signals'])

In [17]:
df

,Twits,Signals
0,$TATASTEEL.NSE,Bullish
1,$TATASTEEL.NSE looks like a fake breakout!! st...,Bullish
2,$TATASTEEL.NSE \nTrail the stoploss to 115.,Null
3,$TATASTEEL.NSE long trade in action \nsafe tr...,Null
4,$TATASTEEL.NSE CMP - 119.80 \nAnother counter...,Bullish
...,...,...
95,$TATASTEEL.NSE finally waking up!!! \nBreakout...,Bullish
96,$TATASTEEL.NSE Cleared lots of resistance zone...,Bullish
97,$TATASTEEL.NSE \nKeep in radar.. looks complet...,Null
98,Traders can go long in $TATASTEEL.NSE if it su...,Bullish
